In [1]:
#Libraries used in the functions below
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import Dash, html, dcc, Input, Output, dash_table, State

# Import plotly
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

In [2]:
#Datasets turned into Pandas DataFrames
dfC = pd.read_csv("dataset/courses.csv")
dfSA=pd.read_csv("dataset/studentAssessment.csv")
dfSI=pd.read_csv("dataset/studentInfo.csv")
dfSR=pd.read_csv("dataset/studentRegistration.csv")
dfVLE=pd.read_csv("dataset/vle.csv")
dfSVLE=pd.read_csv("dataset/studentVle.csv")
dfA=pd.read_csv("dataset/assessments.csv")
dfP=pd.read_csv("dataset/generalized_predicts.csv")

In [3]:
dfC['Course_Section'] = dfC[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
course_section=list(dfC['Course_Section'] )
dfP['Course_Section'] = dfP[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)

In [4]:
dfSI['Course_Section'] = dfSI[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
def roster(course_section,sort_type):
    dfRoster=pd.merge(dfSI,dfP, on=['id_student','Course_Section'])
    dfClassRoster=dfRoster[dfRoster['Course_Section']==course_section]
    if sort_type=='Numerical':
        dfClassRoster=dfClassRoster.sort_values('id_student')
    else:
        dfClassRoster=dfClassRoster.sort_values('probability',ascending=False)
    return list(dfClassRoster['id_student'])

In [5]:
dfA['Course_Section'] = dfA[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
resource_dict = dict(zip(dfVLE['id_site'], dfVLE['activity_type']))
df_resource = pd.DataFrame(dfVLE[['id_site','activity_type']])
def classCode(Module,Presentation):
    dfCode=dfSVLE[dfSVLE['code_module']==Module]
    dfCode=dfCode[dfCode['code_presentation']==Presentation]
    return dfCode

In [64]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = dfC['Course_Section']

# Step 1
df_sa_with_due_dates = dfSA.merge(dfA[['id_assessment','date','code_module','code_presentation','weight']], left_on='id_assessment', right_on='id_assessment')
#dfSI[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
df_sa_with_due_dates['Course_Section'] = df_sa_with_due_dates[['code_module','code_presentation']].apply(lambda x: '-'.join(x), axis=1)
df_si_with_preds =pd.merge(dfSI,dfP, on=['id_student','Course_Section'])
student_info_table_cols = ['id_student','Predicted Fail/Withdraw %', '# of Assignments Completed','Current Assessment weighted % (out of due assignments)', 'Prior assessment average (unweighted)','Most recent assignment score']

app.title = "OU Dashboard"
app.layout = html.Div([
                        html.H1("Instructor Interface/Dashboard",style={'textAlign':'center', 'paddingTop':'1%'}),
                        html.Hr(),
                        html.Div([
                            #html.H1("Instructor Interface/Dashboard"),
                            #html.H5("Select a Module-Presentation"),
                            html.Div([
                                html.Div(["Module-presentation:"], style={'float':'left','margin':'0px','display':'inline-block','fontSize':'20px','fontWeight':'light'}),
                                html.Div(
                                    [dcc.Dropdown(
                                        id='CourseSection',
                                        options=[{'label': i, 'value': i} for i in available_indicators1],
                                        value=available_indicators1[0],
                                        #style={'display':'inline-block','float':'left'}
                                    )],style={'minWidth':'150px','display':'inline-block'}
                                ),
                                
                            ],style={'display':'inline-block'}),
                            
                            html.H5(id='all_student_table_title'),
                            dash_table.DataTable(
                                id="student_info_table",
                                columns=[{"name": i, "id": i} for i in student_info_table_cols],
                                style_cell={
                                        'minWidth': '90px', 
                                        'width': '90px',
                                        'maxWidth': '90px',
                                        'overflow': 'hidden',
                                        'textOverflow': 'ellipsis'},
                                fixed_rows={'headers': True},
                                style_table={'height':'300px'},
                                style_data_conditional=[{
                                    'if':{'row_index':'odd'},
                                    'backgroundColor':'rgb(235,235,235)',
                                },
                                {
                                    'if':{
                                        'filter_query':'{Predicted Fail/Withdraw %} > 0.8'
                                    },
                                    'backgroundColor':'rgb(255,204,203)',
                                    'fontWeight':'bold'
                                }],
                                style_header={'whiteSpace':'normal'},
                                filter_action="native",
                                sort_action="native",
                                sort_mode="multi",
                                data=[]
                                ),
                            dcc.RadioItems(
                                ['Assessment', 'Resources'],
                                'Assessment',
                                id='Graph-Type',
                                inline=True),
                            dcc.Graph(id='GraphMean'),
                            dcc.Graph(id='GraphMedian'),
                                dcc.Slider(
                                    0,
                                    max(dfA['date']),
                                    #200,
                                    step=None,
                                    id='Day',
                                    value=60),
                            html.Div(id="final_table")
                        ],style={'width': '49%','display': 'inline-block'}),
                        #html.Div([], style={'width':'2%', 'display':'inline-block'}),
                        html.Div([
                            #html.H6("Sort by:"),
                            #dcc.RadioItems(
                            #    ['Numerical', 'Failure Rate'],
                            #     id='Sort_by',
                            #     value='Numerical',
                            #    inline=True),
                            #dcc.Dropdown(id='StudentID'),
                            html.Div(id='student_id_msg'),
                            #html.Div(id="StudentID"),
                            html.H6(id='single_student_table_title'),
                            #html.Div(id="Student_table"),
                            dcc.Textarea(
                                id='Reccomendations',
                                value='Recommend actions would go here!',
                                style={'width': '100%', 'height': 100},),
                        ], style={'width': '49%','display': 'inline-block','float': 'right'})
            ], style={'width':'95%','margin':'auto'})

# @app.callback(
#     Output('StudentID', 'options'),
#     Input('CourseSection', 'value'),Input('Sort_by','value'))
# def set_course_options(selected_course,Sort_by):
#     return roster(selected_course,Sort_by)
@app.callback(
    Output('student_info_table','active_cell'),
    Output('student_info_table','selected_cells'),
    Input('CourseSection','value')
)
def reset_selection(course_section):
    return None, []

@app.callback(
    Output('all_student_table_title','children'),
    Input('CourseSection','value'),
    Input('Day','value')
)
def set_all_student_table_title(course_section, day):
    return "{0}'s student information as of day {1}".format(course_section, day)

@app.callback(
    Output('single_student_table_title','children'),
    Input('student_info_table','active_cell'),
    Input('student_info_table','derived_virtual_data')
)
def set_single_student_table_title(active_cell, data):
    id_student = get_id_student(active_cell, data)
    if id_student is not None:
        return "Student {0}'s Information".format(id_student)
    return None

def get_id_student(active_cell, data):
    if active_cell:
        return data[active_cell['row']]['id_student']
    elif data:
        return data[0]['id_student']
    else:    
        return None

@app.callback(
    Output('student_id_msg', 'children'),
    #Input('StudentID', 'options'),
    Input('student_info_table','active_cell'),
    Input('student_info_table','derived_virtual_data'),
    #Input('CourseSection','value')
    )
def set_id_value(active_cell, data):
    id_student = get_id_student(active_cell, data)
    if id_student is not None:
        return "Selected student: {0}".format(id_student)
    return "No data availabe - remove filter and select a student"

# @app.callback(
#     Output('student_id_combined_msg','children'),
#     Input('StudentID','children')
# )
# def set_student_id_msg(student_id):
#     if student_id:
#         return ["Selected student: ", student_id]
#     else:
#         return ["Select a student from the table to see their information!"]

@app.callback(
    Output('student_info_table', 'data'),
    Input('CourseSection', 'value'),Input('Day','value')
)
def update_student_info_table(course_section, day):
    #df_studentInfo = dfSI[dfSI['Course_Section']==course_section]
    df_roster_this_course = pd.DataFrame(df_si_with_preds[df_si_with_preds['Course_Section']==course_section])
    
    #df_sa_with_due_dates
    df_sa_this_course = df_sa_with_due_dates[df_sa_with_due_dates['Course_Section']==course_section]
    df_student_assessments = df_roster_this_course.merge(df_sa_this_course, left_on='id_student',right_on='id_student')
    #print(df_student_assessments.columns)
    
    # filter out assignments due after day
    df_student_assessments = df_student_assessments[df_student_assessments.date < day]
    # filter out assignments submitted after day
    df_student_assessments = df_student_assessments[df_student_assessments.date_submitted < day]

    # calculate total possible scores:
    total_possible_score = dfA[dfA.date < day]
    total_possible_score = total_possible_score[total_possible_score['Course_Section'] == course_section]
    # only count TMAs
    total_possible_score = np.sum(total_possible_score[total_possible_score['assessment_type']=='TMA']['weight'])
    print(total_possible_score)

    # calculate each student's scores
    df_student_assessments['weighted_score'] = df_student_assessments['score'] * df_student_assessments['weight']
    df_student_scores = df_student_assessments.groupby(['id_student']).sum().reset_index()[['id_student','weighted_score']]
    df_student_scores['pct_total'] = np.round(df_student_scores['weighted_score']/total_possible_score,1)

    most_rec_scores = get_most_recent_assignment_scores(dfSA, dfA, dfSI, course_section, day)
    # calculate avg before the last turned in assignment
    #most_rec_scores.loc[most_rec_scores['num_assignments_submitted']==1,'avg_before'] = most_rec_scores.loc[most_rec_scores['num_assignments_submitted']==1,'stu_avg_score']
    no_assignments_submitted_msg = "No prior assignments submitted"
    most_rec_scores.loc[most_rec_scores['num_assignments_submitted']<=1,'avg_before'] = no_assignments_submitted_msg
    most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'avg_before'] = \
        (most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'stu_avg_score']*most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'num_assignments_submitted'] -
        most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'most_rec_score'])/(most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'num_assignments_submitted'] -1)
    # add column indicating how many asisgnments turned in vs total
    most_rec_scores['assignments_completed_frac'] = most_rec_scores['num_assignments_submitted'].astype(str) + '/' + most_rec_scores['num_assignments_due'].astype(str)

    num_assignments_due = np.max(most_rec_scores['num_assignments_due'])
    if pd.isna(num_assignments_due):
        num_assignments_due = 0

    # Columns we want:
    #['id_student','Predicted Fail/Withdraw %', 'Current Assessment %\n(out of due assignments)']
    df_final = df_roster_this_course.merge(df_student_scores, how='left', left_on='id_student',right_on='id_student')[['id_student','probability','pct_total']]
    df_final = df_final.merge(most_rec_scores, how='left',left_on='id_student',right_on='id_student')[list(df_final.columns) + ['avg_before','most_rec_score', 'assignments_completed_frac']]
    #df_final.columns = student_info_table_cols
    # columns: ['id_student',
    #           'Predicted Fail/Withdraw %', 
    #           '# of Assignments Completed',
    #           'Current Assessment weighted % (out of due assignments)', 
    #           'Prior assessment average (unweighted)',
    #           'Most recent assignment score']
    df_final = df_final[['id_student','probability','assignments_completed_frac','pct_total','avg_before','most_rec_score']]
    
    # fill-in blanks
    df_final.loc[df_final['pct_total'].isna(),['assignments_completed_frac','pct_total','avg_before','most_rec_score']] = [
        '0/{0}'.format(num_assignments_due),0,no_assignments_submitted_msg,"Nothing submitted"
    ]

    # if less than 2 assignments turned in - don't show predictions
    if num_assignments_due < 2:
        df_final['probability'] = 'Too early to predict'
    
    df_final.columns = student_info_table_cols
    return df_final.to_dict("records")

    
def get_most_recent_assignment_scores(df_sa, df_a, df_si, course_section, day):
    
    # filter out student assessments submitted on day or after
    df_sa_filtered = df_sa[df_sa['date_submitted'] < day]

    # change any NaNs in student assessment table to 0
    df_sa_filtered.loc[df_sa_filtered['score'].isna(),['score']] = 0
    
    # join with student info - in case student has not submitted any assignments yet...
    student_info = df_si[df_si['Course_Section']==course_section]
    df_sa_filtered = student_info[['id_student']].merge(df_sa_filtered, how='left',left_on='id_student',right_on='id_student')
    
    # filter out assignments with due date on day or after
    df_a_filtered = df_a[(df_a['date']<day) & (df_a['Course_Section']==course_section)]
    
    # calc total # of assignments due
    num_assignments_submittable = df_a_filtered['id_assessment'].count()

    # join to remove any early-submitted assignments from being shown
    df_merged = df_sa_filtered.merge(df_a_filtered, left_on='id_assessment', right_on='id_assessment').sort_values(by=['date_submitted','date'],ascending=[False,False])
    
    # count the number of assignments turned in by student
    num_assignments = df_merged.groupby(['id_student'])['id_assessment'].count().reset_index()[['id_student','id_assessment']].rename(columns={'id_assessment':'num_assignments_submitted'})
    
    # return the average for all assignments turned in by student
    assignment_avg = df_merged.groupby(['id_student'])['score'].mean().reset_index()[['id_student','score']].rename(columns={'score':'stu_avg_score'})

    # get the score of last assignment turned in by the student (for comparison against their previous average)
    to_ret = df_merged.loc[df_merged.groupby(['id_student'])['date_submitted'].idxmax(),['id_student','id_assessment','score']].rename(columns={'score':'most_rec_score'})

    # combine to single df
    to_ret = to_ret.merge(num_assignments, left_on='id_student', right_on='id_student')[['id_student','id_assessment','most_rec_score','num_assignments_submitted']]
    to_ret = to_ret.merge(assignment_avg, left_on='id_student', right_on='id_student')[list(to_ret.columns) + ['stu_avg_score']]
    to_ret['num_assignments_due'] = num_assignments_submittable
    return to_ret 

#def get_avg_score_by_student(df_sa, df_a, course_section, day)

@app.callback(
    Output('GraphMean', 'figure'),Input('CourseSection', 'value'),Input('student_info_table','active_cell'),
    Input('student_info_table','derived_virtual_data'),#Input('StudentID', 'value'),
    Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, active_cell, data, type1,day1): 
    StudentID1 = get_id_student(active_cell, data)
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfSA1=dfSA[dfSA['date_submitted']<=day1+7]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        dictMean={}
        for assess in assessment:
            dfSAMean=dfSA1[dfSA1['id_assessment']==assess]
            dictMean[assess]=np.mean(dfSAMean['score'])
        dfStudentScore=dfSA1[dfSA1['id_student']==StudentID1]
        # add in filter for only assessments from current section
        dfStudentScore=dfStudentScore[dfStudentScore['id_assessment'].isin(assessment)]
        dfStudentScore=dfStudentScore[['id_assessment','score']]
        dfMean = pd.DataFrame(list(dictMean.items()), columns = ['id_assessment','score'])
        dfMean['type']='Mean'
        dfStudentScore['type']='Student'
        dfTogether=pd.concat([dfStudentScore, dfMean], ignore_index=True,axis=0)
        dfTogether['id_assessment'] = dfTogether.id_assessment.astype(str)
        dfTogether=dfTogether.dropna()
        figure = px.bar(dfTogether, x="id_assessment", y="score", color="type", barmode="group",
                title="Comparison of Students Scores and Class Averages")
        return figure
    else:
        dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
        dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
        dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
        student_count=len(dfCode['id_student'].unique())
        dfCode=dfCode.replace({"id_site": resource_dict})
        dfResource=dfCode.groupby(['id_site']).sum()
        dfResource['sum_click']=dfResource['sum_click']/student_count
        dfResource=dfResource.reset_index()
        dfStudent=dfCode[dfCode['id_student']==StudentID1]
        dfStudent=dfStudent.groupby(['id_site']).sum()
        dfStudent=dfStudent.reset_index()
        dfStudent['type']='Student'
        dfResource['type']='Mean'
        dfTogether1=pd.concat([dfStudent, dfResource], ignore_index=True,axis=0)
        figure2 = px.bar(dfTogether1, x="id_site", y="sum_click", color="type", barmode="group",
            title="Comparison of Students Resource Usage and Class Averages")
        return figure2

@app.callback(
    Output('GraphMedian', 'figure'),Input('CourseSection', 'value'),Input('student_info_table','active_cell'),
    Input('student_info_table','derived_virtual_data'),#Input('StudentID', 'value'),
    Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, active_cell, data,type1,day1): 
    StudentID1 = get_id_student(active_cell, data)
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfSA1=dfSA[dfSA['date_submitted']<=day1+7]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        dictMedian={}
        for assess in assessment:
            dfSAMedian=dfSA1[dfSA1['id_assessment']==assess]
            dfSAMedian=dfSAMedian.dropna()
            dictMedian[assess]=np.median(dfSAMedian['score'])
        dfStudentScore=dfSA1[dfSA1['id_student']==StudentID1]
        # add in filter for only assessments from current section
        dfStudentScore=dfStudentScore[dfStudentScore['id_assessment'].isin(assessment)]
        dfStudentScore=dfStudentScore[['id_assessment','score']]
        dfMedian = pd.DataFrame(list(dictMedian.items()), columns = ['id_assessment','score'])
        dfMedian['type']='Median'
        dfStudentScore['type']='Student'
        dfTogether=pd.concat([dfStudentScore, dfMedian], ignore_index=True,axis=0)
        dfTogether['id_assessment'] = dfTogether.id_assessment.astype(str)
        dfTogether=dfTogether.dropna()
        figure = px.bar(dfTogether, x="id_assessment", y="score", color="type", barmode="group",
                title="Comparison of Students Scores and Class Medians")
        return figure
    else:
        dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
        dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
        dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
        dfCode=dfCode.replace({"id_site": resource_dict})
        dfClass=dfCode.groupby(['id_site','id_student']).sum()
        dfClass=dfClass.reset_index()
        resourceType=dfClass['id_site'].unique()
        resourceMedian={}
        for resource in resourceType:
            dfResourceMedian=dfClass[dfClass['id_site']==resource]
            resourceMedian[resource]=np.median(dfResourceMedian['sum_click'])
        dfMedian = pd.DataFrame(list(resourceMedian.items()), columns = ['id_site','sum_click'])
        dfStudent=dfCode[dfCode['id_student']==StudentID1]
        dfStudent=dfStudent.groupby(['id_site']).sum()
        dfStudent=dfStudent.reset_index()
        dfStudent['type']='Student'
        dfMedian['type']='Median'
        dfTogether1=pd.concat([dfStudent, dfMedian], ignore_index=True,axis=0)
        figure2 = px.bar(dfTogether1, x="id_site", y="sum_click", color="type", barmode="group",
            title="Comparison of Students Resource Usage and Class Medians")
        return figure2

@app.callback(
    Output('final_table', 'children'),Input('CourseSection', 'value'),Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, type1, day1):
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfSA1=dfSA[dfSA['date_submitted']<=day1+7]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        trial1=[]
        for assess in assessment:
            dfSAScore=dfSA1[dfSA1['id_assessment']==assess]
            trial1.append([assess]+list(dfSAScore['score'].describe().round(2)))
        data1=pd.DataFrame(trial1,columns=['Assessment','Count','Mean','STD','Min','25%','50%','75%','Max'])
        return [dash_table.DataTable(
                id='Summary',
                columns=[{"name": i, "id": i} for i in data1.columns],
                data=data1.to_dict('records'),
            )]
    else:
        dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
        dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
        dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
        dfCode=dfCode.replace({"id_site": resource_dict})
        dfClass=dfCode.groupby(['id_site','id_student']).sum()
        dfClass=dfClass.reset_index()
        resourceType=dfClass['id_site'].unique()
        trial2=[]
        for idSite in resourceType:
            dfClick=dfClass[dfClass['id_site']==idSite]
            trial2.append([idSite]+list(dfClick['sum_click'].describe().round(2)))
        data2=pd.DataFrame(trial2,columns=['Resource','Count','Mean','STD','Min','25%','50%','75%','Max'])
        return [dash_table.DataTable(
                id='Summary',
                columns=[{"name": i, "id": i} for i in data2.columns],
                data=data2.to_dict('records'),
            )]

# @app.callback(
#    Output('Student_table', 'children'),Input('CourseSection', 'value'),Input('StudentID', 'value'),
#    Input('Graph-Type','value'),Input('Day','value'))
# def update_figure(CourseSection1, StudentID1, type1, day1):
    # dfA1=dfA[dfA['date']<=day1]
    # dfSA1=dfSA[dfSA['date_submitted']<=day1]
    # dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
    # assessment=list(dfCourse_Assessments['id_assessment'])
    # median=[]
    # mean=[]
    # STDEV=[]
    # for assess in assessment:
    #     dfSAMedian=dfSA1[dfSA1['id_assessment']==assess]
    #     dfSAMedian=dfSAMedian.dropna()
    #     median.append(np.median(dfSAMedian['score']).round(2))
    #     mean.append(np.mean(dfSAMedian['score']).round(2))
    #     STDEV.append(np.std(dfSAMedian['score']).round(2))
    # dfStudentScore=dfSA1[dfSA1['id_student']==StudentID1]
    # dfStudentScore=dfStudentScore[['id_assessment','score']]
    # dfStudentScore['Difference (Score-Mean)']=(dfStudentScore['score']-mean).round(2)
    # dfStudentScore['Difference (Score-Median)']=(dfStudentScore['score']-median).round(2)
    # dfStudentScore.columns=['ID','Student','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)']
    # dfStudentScore['STDEV']=STDEV

    # dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
    # dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
    # dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
    # dfCode=dfCode.replace({"id_site": resource_dict})
    # dfClass=dfCode.groupby(['id_site','id_student']).sum()
    # dfClass=dfClass.reset_index()
    # resourceType=dfClass['id_site'].unique()
    # resourceMedian={}
    # resourceMean=[]
    # resourceSTDEV=[]
    # for resource in resourceType:
    #     dfResourceMedian=dfClass[dfClass['id_site']==resource]
    #     resourceMedian[resource]=(np.median(dfResourceMedian['sum_click']).round(2))
    #     resourceMean.append(np.mean(dfResourceMedian['sum_click']).round(2))
    #     resourceSTDEV.append(np.std(dfResourceMedian['sum_click']).round(2))
    # dfStudent=dfCode[dfCode['id_student']==StudentID1]
    # dfStudent=dfStudent.groupby(['id_site']).sum()
    # dfStudent=dfStudent.reset_index()
    # ResourceList=[resourceMedian.keys(),resourceMedian.values()]
    # dfResourceList=pd.DataFrame(ResourceList).T
    # dfResourceList['Mean']=resourceMean
    # dfResourceList['STDEV']=resourceSTDEV
    # dfResourceList.columns=['id_site','Median','Mean','STDEV']
    # dfStudentResource=pd.merge(dfResourceList, dfStudent, on='id_site',how='left')
    # dfStudentResource=dfStudentResource.fillna(0)
    # # del dfStudentResource['id_student']
    # # del dfStudentResource['date']
    # dfStudentResource['Median']=(dfStudentResource['sum_click']-dfStudentResource['Median']).round(2)
    # dfStudentResource['Mean']=(dfStudentResource['sum_click']-dfStudentResource['Mean']).round(2)
    # dfStudentResource=dfStudentResource[['id_site','sum_click','Median','Mean','STDEV']]
    # dfStudentResource.columns=['ID','Student','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','STDEV']
    # dfTogetherStudent=pd.concat([dfStudentScore, dfStudentResource], ignore_index=True,axis=0)
    # count=0
    # for thing in dfTogetherStudent['STDEV']:
    #     if dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0:
    #         if dfTogetherStudent['Difference (Sum_Click - Mean)'][count]<=0.5*thing:
    #             dfTogetherStudent['STDEV'][count]='Average'
    #         elif dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>=2*thing:
    #             dfTogetherStudent['STDEV'][count]='Well Above Average'
    #         elif dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0.5*thing:
    #             dfTogetherStudent['STDEV'][count]='Above Average'
    #         count=count+1
    #     else:
    #         if -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]<=0.5*thing:
    #             dfTogetherStudent['STDEV'][count]='Average'
    #         elif -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>=2*thing:
    #             dfTogetherStudent['STDEV'][count]='Well Below Average'
    #         elif -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0.5*thing:
    #             dfTogetherStudent['STDEV'][count]='Below Average'
    #         count=count+1
    # dfTogetherStudent.columns=['Assessment/Resource','Student Score/Clicks','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','Performance']
    # return [dash_table.DataTable(
    #        id='Summary1',
    #        columns=[{"name": i, "id": i} for i in dfTogetherStudent.columns],
    #        data=dfTogetherStudent.to_dict('records'),
    #        )]
    #return None
    
app.run_server(mode="external", port = 8101, debug=False)

Dash app running on http://127.0.0.1:8101/


30.0
10.0
10.0
30.0


In [48]:
#get_most_recent_assignment_scores(dfSA, dfA, 'AAA-2013J',10)
#dfA['id_assessment'].count()
test_ = pd.Series([None, np.nan]).isna()
test_

0    True
1    True
dtype: bool

In [15]:
# Test performance
# dfSVLE1=dfSVLE[dfSVLE['date']<=60]
# dfCode=dfSVLE1[dfSVLE1['code_module']=="AAA-2013J"[:3]]
# dfCode=dfCode[dfCode['code_presentation']=="AAA-2013J"[4:]]
# dfCode=dfCode.replace({"id_site": resource_dict})
# dfClass=dfCode.groupby(['id_site','id_student']).sum()
# dfClass=dfClass.reset_index()
# resourceType=dfClass['id_site'].unique()
# resourceType

array(['dataplus', 'forumng', 'glossary', 'homepage', 'oucollaborate',
       'oucontent', 'resource', 'subpage', 'url'], dtype=object)

In [8]:
# test performance
# dfSVLE1=dfSVLE[dfSVLE['date']<=60]
# dfCode=dfSVLE1[dfSVLE1['code_module']=="AAA-2013J"[:3]]
# dfCode=dfCode[dfCode['code_presentation']=="AAA-2013J"[4:]]

# merged = dfCode.merge(df_resource, left_on='id_site', right_on='id_site')

# dfCode.loc[:,'id_site']=merged.loc[:,'activity_type']
# dfClass=dfCode.groupby(['id_site','id_student']).sum()
# dfClass=dfClass.reset_index()

# resourceType=dfClass['id_site'].unique()
# resourceType

array(['dataplus', 'forumng', 'glossary', 'homepage', 'oucollaborate',
       'oucontent', 'resource', 'subpage', 'url'], dtype=object)

In [57]:
#dfSA[dfSA.id_student == 721259]

,id_assessment,id_student,date_submitted,is_banked,score
215,1752,721259,22,0,NaN
1656,1758,721259,19,0,65.0


In [58]:
df_sa_with_due_dates[(df_sa_with_due_dates.id_assessment == 721259) & (df_sa_with_due_dates.Course_Section=='AAA-2013J')].describe()

,id_assessment,id_student,date_submitted,is_banked,score,date,weight
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_sa_with_due_dates[df_sa_with_due_dates.id_student == 721259]

,id_assessment,id_student,date_submitted,is_banked,score,date,code_module,code_presentation,weight,Course_Section
215,1752,721259,22,0,NaN,19.0,AAA,2013J,10.0,AAA-2013J
1656,1758,721259,19,0,65.0,19.0,AAA,2014J,10.0,AAA-2014J


In [80]:
# issue with filtered data selection: 
# Filter data, then select student
# selected student is incorrect
df_sa_with_due_dates[df_sa_with_due_dates.id_student == 157568]

,id_assessment,id_student,date_submitted,is_banked,score,date,code_module,code_presentation,weight,Course_Section
58,1752,157568,18,0,79.0,19.0,AAA,2013J,10.0,AAA-2013J
555,1753,157568,66,0,81.0,54.0,AAA,2013J,20.0,AAA-2013J
894,1754,157568,138,0,74.0,117.0,AAA,2013J,20.0,AAA-2013J
1080,1755,157568,181,0,60.0,166.0,AAA,2013J,20.0,AAA-2013J
1504,1756,157568,215,0,59.0,215.0,AAA,2013J,30.0,AAA-2013J
